In [1]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
from tqdm import tqdm

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

2023-06-29 14:53:46.535909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 14:53:47.552470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-29 14:53:47.552614: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
      i_image = Image.open(image_path)
      if i_image.mode != "RGB":
          i_image = i_image.convert(mode="RGB")

      images.append(i_image)
      
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  i_image.close()
  return preds

In [3]:
import glob
import pandas as pd
from tqdm import tqdm

img_to_text = {'img_path':[], 'prompt':[]}
file_path = f'/home/gi2570/coco/images/train2017/'
list_images= glob.glob(file_path + '*.jpg')

img_to_text['img_path'].extend(list_images)
for img in tqdm(list_images):
    img_to_text['prompt'].append(predict_step([img]))
pd.DataFrame(img_to_text)

  8%|▊         | 9395/118287 [49:59<8:44:31,  3.46it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 13%|█▎        | 14914/118287 [1:17:48<9:12:36,  3.12it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 20%|█▉        | 23508/118287 [2:00:04<7:18:12,  3.60it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable

,img_path,prompt
0,/home/gi2570/coco/images/train2017/00000044379...,[a bedroom with a bed and a dresser]
1,/home/gi2570/coco/images/train2017/00000032752...,[a black and white horse standing next to a cr...
2,/home/gi2570/coco/images/train2017/00000028883...,[a cut in half sandwich sitting on top of a cu...
3,/home/gi2570/coco/images/train2017/00000052356...,[two dogs sitting in the back of a pick up truck]
4,/home/gi2570/coco/images/train2017/00000004838...,[a white horse standing on top of a sandy beach]
...,...,...
118282,/home/gi2570/coco/images/train2017/00000004162...,[a brown and white dog laying on the ground wi...
118283,/home/gi2570/coco/images/train2017/00000057165...,[two women in a kitchen with a plate of food]
118284,/home/gi2570/coco/images/train2017/00000041587...,[a piece of cake sitting on top of a paper plate]
118285,/home/gi2570/coco/images/train2017/00000000883...,[a toilet sitting in the middle of a forest]


In [5]:
pd.DataFrame(img_to_text)

,img_path,prompt
0,/home/gi2570/coco/images/train2017/00000044379...,[a bedroom with a bed and a dresser]
1,/home/gi2570/coco/images/train2017/00000032752...,[a black and white horse standing next to a cr...
2,/home/gi2570/coco/images/train2017/00000028883...,[a cut in half sandwich sitting on top of a cu...
3,/home/gi2570/coco/images/train2017/00000052356...,[two dogs sitting in the back of a pick up truck]
4,/home/gi2570/coco/images/train2017/00000004838...,[a white horse standing on top of a sandy beach]
...,...,...
118282,/home/gi2570/coco/images/train2017/00000004162...,[a brown and white dog laying on the ground wi...
118283,/home/gi2570/coco/images/train2017/00000057165...,[two women in a kitchen with a plate of food]
118284,/home/gi2570/coco/images/train2017/00000041587...,[a piece of cake sitting on top of a paper plate]
118285,/home/gi2570/coco/images/train2017/00000000883...,[a toilet sitting in the middle of a forest]


In [10]:
df_tmp = pd.DataFrame(img_to_text)
df_tmp.to_csv("/home/gi2570/coco/img_caption/train2017_df.csv", index = False)

In [6]:
import glob
import pandas as pd
coco_lst = ['test', 'unlabeled', 'val']
for i in coco_lst:
    img_to_text = {'img_path':[], 'prompt':[]}
    file_path = f'/home/gi2570/coco/images/{i}2017/'
    list_images= glob.glob(file_path + '*.jpg')

    img_to_text['img_path'].extend(list_images)
    for img in tqdm(list_images):
        img_to_text['prompt'].append(predict_step([img]))
        
    df_tmp = pd.DataFrame(img_to_text)
    df_tmp.to_csv(f"/home/gi2570/coco/img_caption/{i}2017_df.csv", index = False)


IsADirectoryError: [Errno 21] Is a directory: '/'